# Generative Question Answering with BERT + Decoder

Ноутбук для обучения генеративной вопросно-ответной системы на основе BERT encoder и Transformer decoder.

**Цель**: обучить модель генерировать ответы на вопросы, используя BERT для кодирования контекста и вопроса, и decoder для генерации ответа.

**Архитектура**: BERT-base-uncased (encoder) + Transformer Decoder → генерация ответов


## Шаг 1: Импорты и настройка


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from datasets import load_dataset
import random
import pytorch_lightning as pl
from transformers import BertModel, BertTokenizerFast
import math

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
pl.seed_everything(42)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")


Seed set to 42


PyTorch version: 2.5.1+cu121
CUDA available: True


## Шаг 2: Загрузка датасета и токенизатора

Используем SQuAD датасет. Для генеративной модели нужен токенизатор, который будет работать и для encoder (BERT) и для decoder (используем тот же BERT токенизатор).


In [2]:
print("Loading SQuAD dataset...")
dataset = load_dataset("squad")

train_data = dataset['train']
val_data = dataset['validation']

# Используем полный датасет
train_contexts = train_data['context']
train_questions = train_data['question']
train_answers = train_data['answers']

val_contexts = val_data['context']
val_questions = val_data['question']
val_answers = val_data['answers']

print(f"Train examples: {len(train_contexts)}")
print(f"Val examples: {len(val_contexts)}")

model_name = 'bert-base-uncased'
tokenizer = BertTokenizerFast.from_pretrained(model_name)

print(f"\nTokenizer loaded: {model_name}")
print(f"Vocab size: {tokenizer.vocab_size}")
print(f"Special tokens: {tokenizer.special_tokens_map}")

sample_question = train_questions[0]
sample_context = train_contexts[0]
sample_answer = train_answers[0]['text'][0]

print(f"\nSample:")
print(f"Question: {sample_question}")
print(f"Answer: {sample_answer}")


Loading SQuAD dataset...
Train examples: 87599
Val examples: 10570

Tokenizer loaded: bert-base-uncased
Vocab size: 30522
Special tokens: {'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}

Sample:
Question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Answer: Saint Bernadette Soubirous


## Шаг 3: Dataset для генеративной модели

Создаем Dataset класс, который токенизирует:
- Input: question + context (для encoder)
- Target: answer (для decoder, с teacher forcing)


In [3]:
class GenerativeQADataset(Dataset):
    
    def __init__(self, contexts, questions, answers, tokenizer, max_encoder_length=384, max_decoder_length=64):
        self.contexts = contexts
        self.questions = questions
        self.answers = answers
        self.tokenizer = tokenizer
        self.max_encoder_length = max_encoder_length
        self.max_decoder_length = max_decoder_length
        
        self.answer_texts = []
        for answer_dict in answers:
            answer_text = answer_dict['text'][0]
            self.answer_texts.append(answer_text)
    
    def __len__(self):
        return len(self.contexts)
    
    def __getitem__(self, idx):
        question = self.questions[idx]
        context = self.contexts[idx]
        answer = self.answer_texts[idx]
        
        # Правильная токенизация encoder input: question + context
        # tokenizer автоматически добавит [CLS] question [SEP] context [SEP]
        encoder_encoded = self.tokenizer(
            question,
            context,
            max_length=self.max_encoder_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        # Токенизируем ответ БЕЗ специальных токенов
        answer_tokens = self.tokenizer.encode(answer, add_special_tokens=False)
        
        # Обрезаем если слишком длинный (оставляем место для [CLS] и [SEP])
        max_answer_len = self.max_decoder_length - 2
        answer_tokens = answer_tokens[:max_answer_len]
        
        # Teacher forcing:
        # decoder_input:  [CLS] word1 word2 ... wordN [PAD] [PAD] ...
        # decoder_target: word1 word2 ... wordN [SEP] [PAD] [PAD] ...
        
        decoder_input_ids = [self.tokenizer.cls_token_id] + answer_tokens
        decoder_target_ids = answer_tokens + [self.tokenizer.sep_token_id]
        
        # Паддинг до max_decoder_length
        pad_len = self.max_decoder_length - len(decoder_input_ids)
        decoder_input_ids = decoder_input_ids + [self.tokenizer.pad_token_id] * pad_len
        decoder_target_ids = decoder_target_ids + [self.tokenizer.pad_token_id] * pad_len
        
        # Attention mask: 1 для реальных токенов, 0 для padding
        decoder_attention_mask = [1] * (len(answer_tokens) + 1) + [0] * pad_len
        
        return {
            'encoder_input_ids': encoder_encoded['input_ids'].squeeze(0),
            'encoder_attention_mask': encoder_encoded['attention_mask'].squeeze(0),
            'decoder_input_ids': torch.tensor(decoder_input_ids, dtype=torch.long),
            'decoder_attention_mask': torch.tensor(decoder_attention_mask, dtype=torch.long),
            'decoder_target_ids': torch.tensor(decoder_target_ids, dtype=torch.long),
            'answer_text': answer,
            'question': question,
            'context': context
        }

train_dataset = GenerativeQADataset(
    train_contexts,
    train_questions,
    train_answers,
    tokenizer,
    max_encoder_length=384,
    max_decoder_length=64
)

val_dataset = GenerativeQADataset(
    val_contexts,
    val_questions,
    val_answers,
    tokenizer,
    max_encoder_length=384,
    max_decoder_length=64
)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Val dataset size: {len(val_dataset)}")

sample = train_dataset[0]
print(f"\nSample from dataset:")
print(f"Encoder input IDs shape: {sample['encoder_input_ids'].shape}")
print(f"Decoder input IDs shape: {sample['decoder_input_ids'].shape}")
print(f"Decoder target IDs shape: {sample['decoder_target_ids'].shape}")
print(f"Answer: {sample['answer_text']}")


Train dataset size: 87599
Val dataset size: 10570

Sample from dataset:
Encoder input IDs shape: torch.Size([384])
Decoder input IDs shape: torch.Size([64])
Decoder target IDs shape: torch.Size([64])
Answer: Saint Bernadette Soubirous


In [4]:
class TransformerDecoderLayer(nn.Module):
    
    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        self.multihead_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)
        
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)
        
    def forward(self, tgt, memory, tgt_mask=None, memory_key_padding_mask=None, tgt_key_padding_mask=None):
        tgt2 = self.self_attn(tgt, tgt, tgt, attn_mask=tgt_mask, key_padding_mask=tgt_key_padding_mask)[0]
        tgt = tgt + self.dropout1(tgt2)
        tgt = self.norm1(tgt)
        
        tgt2 = self.multihead_attn(tgt, memory, memory, key_padding_mask=memory_key_padding_mask)[0]
        tgt = tgt + self.dropout2(tgt2)
        tgt = self.norm2(tgt)
        
        tgt2 = self.linear2(self.dropout(F.relu(self.linear1(tgt))))
        tgt = tgt + self.dropout3(tgt2)
        tgt = self.norm3(tgt)
        
        return tgt

class TransformerDecoder(nn.Module):
    
    def __init__(self, d_model=768, nhead=12, num_layers=6, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        self.layers = nn.ModuleList([
            TransformerDecoderLayer(d_model, nhead, dim_feedforward, dropout)
            for _ in range(num_layers)
        ])
        
    def forward(self, tgt, memory, tgt_mask=None, memory_key_padding_mask=None, tgt_key_padding_mask=None):
        output = tgt
        for layer in self.layers:
            output = layer(output, memory, tgt_mask, memory_key_padding_mask, tgt_key_padding_mask)
        return output

print("Transformer Decoder created")


Transformer Decoder created


## Шаг 5: BERT + Decoder модель

Создаем полную модель: BERT encoder + Transformer decoder + output projection для генерации токенов.


In [5]:
class BERTGenerativeQA(nn.Module):
    
    def __init__(self, model_name='bert-base-uncased', vocab_size=30522, d_model=768, nhead=12, 
                 num_decoder_layers=6, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        
        self.encoder = BertModel.from_pretrained(model_name)
        self.d_model = d_model
        
        self.decoder_embedding = nn.Embedding(vocab_size, d_model)
        self.decoder_pos_encoding = PositionalEncoding(d_model, dropout)
        self.decoder = TransformerDecoder(d_model, nhead, num_decoder_layers, dim_feedforward, dropout)
        
        self.output_projection = nn.Linear(d_model, vocab_size)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, encoder_input_ids, encoder_attention_mask, decoder_input_ids, decoder_attention_mask=None):
        encoder_outputs = self.encoder(input_ids=encoder_input_ids, attention_mask=encoder_attention_mask)
        encoder_hidden_states = encoder_outputs.last_hidden_state
        
        decoder_embeds = self.decoder_embedding(decoder_input_ids) * math.sqrt(self.d_model)
        decoder_embeds = self.decoder_pos_encoding(decoder_embeds)
        
        tgt_mask = self.generate_square_subsequent_mask(decoder_input_ids.size(1)).to(decoder_input_ids.device)
        
        decoder_outputs = self.decoder(
            decoder_embeds.transpose(0, 1),
            encoder_hidden_states.transpose(0, 1),
            tgt_mask=tgt_mask,
            memory_key_padding_mask=(encoder_attention_mask == 0),
            tgt_key_padding_mask=(decoder_attention_mask == 0) if decoder_attention_mask is not None else None
        )
        
        decoder_outputs = decoder_outputs.transpose(0, 1)
        logits = self.output_projection(decoder_outputs)
        
        return logits
    
    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

class PositionalEncoding(nn.Module):
    
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)

print("BERT + Decoder model created")


BERT + Decoder model created


## Шаг 6: Lightning модуль

Создаем Lightning модуль для обучения генеративной модели.


In [6]:
class GenerativeQALightningModule(pl.LightningModule):
    
    def __init__(self, model_name='bert-base-uncased', vocab_size=30522, d_model=768, 
                 nhead=12, num_decoder_layers=3, dim_feedforward=2048, dropout=0.1, 
                 encoder_lr=2e-5, decoder_lr=1e-4, warmup_epochs=2):
        super().__init__()
        self.save_hyperparameters()
        
        self.model = BERTGenerativeQA(
            model_name=model_name,
            vocab_size=vocab_size,
            d_model=d_model,
            nhead=nhead,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout
        )
        self.encoder_lr = encoder_lr
        self.decoder_lr = decoder_lr
        self.warmup_epochs = warmup_epochs
        self.tokenizer = tokenizer
        self.vocab_size = vocab_size
        
        self.criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
        
    def forward(self, encoder_input_ids, encoder_attention_mask, decoder_input_ids, decoder_attention_mask=None):
        return self.model(encoder_input_ids, encoder_attention_mask, decoder_input_ids, decoder_attention_mask)
    
    def training_step(self, batch, batch_idx):
        encoder_input_ids = batch['encoder_input_ids']
        encoder_attention_mask = batch['encoder_attention_mask']
        decoder_input_ids = batch['decoder_input_ids']
        decoder_attention_mask = batch['decoder_attention_mask']
        decoder_target_ids = batch['decoder_target_ids']
        
        logits = self(encoder_input_ids, encoder_attention_mask, decoder_input_ids, decoder_attention_mask)
        
        logits = logits.view(-1, self.vocab_size)
        targets = decoder_target_ids.view(-1)
        
        loss = self.criterion(logits, targets)
        
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        encoder_input_ids = batch['encoder_input_ids']
        encoder_attention_mask = batch['encoder_attention_mask']
        decoder_input_ids = batch['decoder_input_ids']
        decoder_attention_mask = batch['decoder_attention_mask']
        decoder_target_ids = batch['decoder_target_ids']
        answer_texts = batch['answer_text']
        
        logits = self(encoder_input_ids, encoder_attention_mask, decoder_input_ids, decoder_attention_mask)
        
        logits = logits.view(-1, self.vocab_size)
        targets = decoder_target_ids.view(-1)
        
        loss = self.criterion(logits, targets)
        
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        
        return loss
    
    def configure_optimizers(self):
        # Разные learning rates для encoder (BERT) и decoder
        encoder_params = list(self.model.encoder.parameters())
        decoder_params = (
            list(self.model.decoder_embedding.parameters()) +
            list(self.model.decoder_pos_encoding.parameters()) +
            list(self.model.decoder.parameters()) +
            list(self.model.output_projection.parameters())
        )
        
        optimizer = torch.optim.AdamW([
            {'params': encoder_params, 'lr': self.encoder_lr},
            {'params': decoder_params, 'lr': self.decoder_lr}
        ], betas=(0.9, 0.999), eps=1e-8, weight_decay=0.01)
        
        total_steps = self.trainer.estimated_stepping_batches
        warmup_steps = int(total_steps * (self.warmup_epochs / self.trainer.max_epochs)) if self.trainer.max_epochs > 0 else int(total_steps * 0.1)
        
        def lr_lambda(current_step):
            if current_step < warmup_steps:
                return float(current_step) / float(max(1, warmup_steps))
            return max(0.0, float(total_steps - current_step) / float(max(1, total_steps - warmup_steps)))
        
        scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)
        
        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': scheduler,
                'interval': 'step',
                'frequency': 1
            }
        }

print("Lightning module created")


Lightning module created


## Шаг 7: Обучение модели

Создаем DataLoaders и запускаем обучение.


In [7]:
train_loader = DataLoader(
    train_dataset,
    batch_size=128,
    shuffle=True,
    num_workers=4
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=4
)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")

lightning_model = GenerativeQALightningModule(
    model_name=model_name,
    vocab_size=tokenizer.vocab_size,
    d_model=768,
    nhead=12,
    num_decoder_layers=3,
    dim_feedforward=2048,
    dropout=0.1,
    encoder_lr=2e-5,   # Меньший lr для предобученного BERT
    decoder_lr=1e-4,   # Больший lr для случайно инициализированного decoder
    warmup_epochs=2
)

trainer = pl.Trainer(
    max_epochs=10,
    accelerator='auto',
    devices=1,
    enable_progress_bar=True,
    log_every_n_steps=50,
    gradient_clip_val=1.0,
    accumulate_grad_batches=1,
    check_val_every_n_epoch=1
)

print("\nStarting training...")
trainer.fit(lightning_model, train_loader, val_loader)
print("\nTraining completed!")


Train batches: 685
Val batches: 331


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L40') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]



Starting training...


Loading `train_dataloader` to estimate number of stepping batches.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | BERTGenerativeQA | 180 M  | train
1 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
180 M     Trainable params
0         Non-trainable params
180 M     Total params
720.113   Total estimated model params size (MB)
51        Modules in train mode
228       Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



## Шаг 8: Генерация ответов

Тестируем модель, генерируя ответы на вопросы из валидационного набора.


In [8]:
def generate_answer(model, encoder_input_ids, encoder_attention_mask, tokenizer, max_length=64, temperature=1.0, top_k=50):
    model.eval()
    device = encoder_input_ids.device
    
    decoder_input_ids = torch.tensor([[tokenizer.cls_token_id]], device=device)
    
    with torch.no_grad():
        for step in range(max_length):
            decoder_attention_mask = torch.ones_like(decoder_input_ids, device=device)
            
            logits = model(encoder_input_ids, encoder_attention_mask, decoder_input_ids, decoder_attention_mask)
            next_token_logits = logits[0, -1, :]
            
            if top_k > 0:
                next_token_logits = next_token_logits / temperature
                top_k_logits, top_k_indices = torch.topk(next_token_logits, min(top_k, next_token_logits.size(0)))
                next_token_logits_filtered = torch.full_like(next_token_logits, float('-inf'))
                next_token_logits_filtered.scatter_(0, top_k_indices, top_k_logits)
                probs = F.softmax(next_token_logits_filtered, dim=-1)
                next_token_id = torch.multinomial(probs, num_samples=1).unsqueeze(0)
            else:
                next_token_id = torch.argmax(next_token_logits, dim=-1).unsqueeze(0).unsqueeze(0)
            
            decoder_input_ids = torch.cat([decoder_input_ids, next_token_id], dim=1)
            
            if next_token_id.item() == tokenizer.sep_token_id or next_token_id.item() == tokenizer.pad_token_id:
                break
    
    generated_ids = decoder_input_ids[0].cpu().tolist()
    answer = tokenizer.decode(generated_ids, skip_special_tokens=True)
    
    if answer.startswith(tokenizer.cls_token):
        answer = answer[len(tokenizer.cls_token):].strip()
    
    return answer

lightning_model.eval()
device = next(lightning_model.parameters()).device
sample_batch = next(iter(val_loader))

print("Testing generation with different strategies:\n")

for i in range(3):
    encoder_input_ids = sample_batch['encoder_input_ids'][i:i+1].to(device)
    encoder_attention_mask = sample_batch['encoder_attention_mask'][i:i+1].to(device)
    
    print(f"\nQuestion: {sample_batch['question'][i]}")
    print(f"Context: {sample_batch['context'][i][:150]}...")
    print(f"Ground Truth: {sample_batch['answer_text'][i]}")
    
    generated_greedy = generate_answer(
        lightning_model.model,
        encoder_input_ids,
        encoder_attention_mask,
        tokenizer,
        max_length=64,
        temperature=1.0,
        top_k=0
    )
    print(f"Generated (greedy): '{generated_greedy}'")
    
    generated_sampled = generate_answer(
        lightning_model.model,
        encoder_input_ids,
        encoder_attention_mask,
        tokenizer,
        max_length=64,
        temperature=0.7,
        top_k=50
    )
    print(f"Generated (sampled, temp=0.7): '{generated_sampled}'")
    
    if not generated_greedy or generated_greedy.strip() == "":
        print("WARNING: Empty answer generated!")


Testing generation with different strategies:


Question: Which NFL team represented the AFC at Super Bowl 50?
Context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football...
Ground Truth: Denver Broncos
Generated (greedy): 'colorado giants'
Generated (sampled, temp=0.7): 'colorado giants'

Question: Which NFL team represented the NFC at Super Bowl 50?
Context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football...
Ground Truth: Carolina Panthers
Generated (greedy): 'colorado giants'
Generated (sampled, temp=0.7): 'dc cubs'

Question: Where did Super Bowl 50 take place?
Context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football...
Ground Truth: Santa Clara, California
Generated (greedy): 'san